# Minimal Effective Pipeline for Price Prediction
Focus: Only what matters

In [1]:
import os
import sys
import re
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from pathlib import Path
from dotenv import load_dotenv
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
import warnings
warnings.filterwarnings('ignore')

project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

from src.utils.seed import seed_everything
from src.training.metrics import smape_np
from src.data.load import read_csvs, make_folds

load_dotenv()

SEED = 42
DEVICE = torch.device(os.getenv("DEVICE", "mps"))
TRAIN_CSV = str(project_root / os.getenv("TRAIN_CSV", "dataset/train.csv"))
TEST_CSV = str(project_root / os.getenv("TEST_CSV", "dataset/test.csv"))
IMG_DIR = project_root / Path(os.getenv("IMG_DIR", "data/processed/images"))
OUT_DIR = project_root / "outputs_minimal"
HF_HOME = project_root / ".hf_cache"

OUT_DIR.mkdir(exist_ok=True)
HF_HOME.mkdir(exist_ok=True)
seed_everything(SEED)

print(f"Device: {DEVICE}")

Device: mps


In [2]:
# Load data
train_df, test_df = read_csvs(TRAIN_CSV, TEST_CSV)
train_df = make_folds(train_df, n_folds=5, seed=SEED)

# Holdout split
y_log = np.log1p(train_df["price"].clip(lower=0.0))
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=SEED)
tr_idx, va_idx = next(sss.split(np.zeros(len(y_log)), pd.qcut(y_log, q=20, duplicates="drop")))
hold_tr = train_df.iloc[tr_idx].copy()
hold_va = train_df.iloc[va_idx].copy()

print(f"Train: {len(train_df)}, Test: {len(test_df)}")
print(f"Holdout: {len(hold_tr)} train, {len(hold_va)} val")

Loading training data from: /Users/adyasrivastava/Downloads/smart_pricing/dataset/train.csv
Loading test data from: /Users/adyasrivastava/Downloads/smart_pricing/dataset/test.csv

Train shape: (75000, 4)
Test shape: (75000, 3)

Price statistics:
  Min: $0.13
  Max: $2796.00
  Mean: $23.65
  Median: $14.00

Images available:
  Train: 75000/75000 (100.0%)
  Test: 75000/75000 (100.0%)

Created 5 stratified folds:
  Fold 0: 15000 samples, mean price: $23.46, median: $14.10
  Fold 1: 15000 samples, mean price: $23.63, median: $13.99
  Fold 2: 15000 samples, mean price: $23.47, median: $14.20
  Fold 3: 15000 samples, mean price: $23.96, median: $13.99
  Fold 4: 15000 samples, mean price: $23.73, median: $14.05
Train: 75000, Test: 75000
Holdout: 63750 train, 11250 val


# Critical Features Only

In [3]:
def extract_smart_features(text):
    """Extract only the most predictive features"""
    text = str(text).lower()
    
    # Quantities (most important!)
    weight_kg = sum([float(m.group(1)) for m in re.finditer(r'(\d+\.?\d*)\s*kg', text)])
    weight_g = sum([float(m.group(1)) for m in re.finditer(r'(\d+\.?\d*)\s*g\b', text)])
    volume_l = sum([float(m.group(1)) for m in re.finditer(r'(\d+\.?\d*)\s*l\b', text)])
    volume_ml = sum([float(m.group(1)) for m in re.finditer(r'(\d+\.?\d*)\s*ml', text)])
    
    # Convert to standard units
    total_g = weight_kg * 1000 + weight_g
    total_ml = volume_l * 1000 + volume_ml
    
    # Pack size
    pack = 1
    for pattern in [r'(\d+)\s*pack', r'(\d+)\s*ct\b']:
        matches = re.findall(pattern, text)
        if matches:
            pack = max(pack, max([int(m) for m in matches]))
    
    # Simple text stats
    words = len(text.split())
    numbers = len(re.findall(r'\d+', text))
    
    # Premium indicator
    premium = int(any(w in text for w in ['organic', 'premium', 'natural', 'gourmet']))
    
    return {
        'weight_g': total_g,
        'volume_ml': total_ml,
        'pack_count': pack,
        'log_weight': np.log1p(total_g),
        'log_volume': np.log1p(total_ml),
        'word_count': words,
        'num_count': numbers,
        'is_premium': premium,
    }

train_df = train_df.join(train_df['catalog_content'].fillna('').apply(extract_smart_features).apply(pd.Series))
test_df = test_df.join(test_df['catalog_content'].fillna('').apply(extract_smart_features).apply(pd.Series))
hold_tr = hold_tr.join(hold_tr['catalog_content'].fillna('').apply(extract_smart_features).apply(pd.Series))
hold_va = hold_va.join(hold_va['catalog_content'].fillna('').apply(extract_smart_features).apply(pd.Series))

feature_cols = ['weight_g', 'volume_ml', 'pack_count', 'log_weight', 
                'log_volume', 'word_count', 'num_count', 'is_premium', 'has_image']

print(f"Using {len(feature_cols)} features")

Using 9 features


# Best Transformer: sentence-transformers/all-mpnet-base-v2

In [ ]:
from transformers import AutoTokenizer, AutoModel
from tqdm import trange

# This is consistently one of the best general-purpose encoders
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"

print(f"Loading: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=str(HF_HOME))
model = AutoModel.from_pretrained(MODEL_NAME, cache_dir=str(HF_HOME)).to(DEVICE)
model.eval()

def mean_pool(output, mask):
    embeddings = output[0]
    mask_expanded = mask.unsqueeze(-1).expand(embeddings.size()).float()
    return torch.sum(embeddings * mask_expanded, 1) / torch.clamp(mask_expanded.sum(1), min=1e-9)

@torch.no_grad()
def encode(texts, batch_size=64):
    all_emb = []
    for i in trange(0, len(texts), batch_size, desc="Encoding"):
        batch = texts[i:i+batch_size]
        enc = tokenizer(batch, padding=True, truncation=True, max_length=128, return_tensors="pt")
        enc = {k: v.to(DEVICE) for k, v in enc.items()}
        out = model(**enc)
        pooled = mean_pool(out, enc['attention_mask'])
        pooled = torch.nn.functional.normalize(pooled, p=2, dim=1)
        all_emb.append(pooled.cpu().numpy())
    return np.vstack(all_emb).astype(np.float32)

# Encode
texts_train = train_df['catalog_content'].fillna('').astype(str).tolist()
texts_test = test_df['catalog_content'].fillna('').astype(str).tolist()
texts_htr = hold_tr['catalog_content'].fillna('').astype(str).tolist()
texts_hva = hold_va['catalog_content'].fillna('').astype(str).tolist()

emb_train = encode(texts_train)
emb_test = encode(texts_test)
emb_htr = encode(texts_htr)
emb_hva = encode(texts_hva)

print(f"Embedding shape: {emb_train.shape}")

Loading: sentence-transformers/all-mpnet-base-v2


Encoding:  10%|▉         | 115/1172 [06:16<55:56,  3.18s/it]  

# LightGBM Only (Fast + Effective)

In [ ]:
try:
    import lightgbm as lgb
except:
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "lightgbm", "-q"])
    import lightgbm as lgb

print("\nTraining LightGBM with 5-fold CV...")

# Combine embeddings + features (no scaling for trees!)
X_train = np.hstack([emb_train, train_df[feature_cols].values]).astype(np.float32)
X_test = np.hstack([emb_test, test_df[feature_cols].values]).astype(np.float32)

oof = np.zeros(len(train_df))
test_pred = np.zeros(len(test_df))

params = {
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'max_depth': -1,
    'min_child_samples': 20,
    'reg_alpha': 0.1,
    'reg_lambda': 1.0,
    'verbose': -1,
    'random_state': SEED,
}

for fold in range(5):
    print(f"\nFold {fold+1}/5:")
    tr_idx = train_df[train_df.fold != fold].index
    va_idx = train_df[train_df.fold == fold].index
    
    # Train in log space
    train_set = lgb.Dataset(X_train[tr_idx], np.log1p(train_df.loc[tr_idx, 'price']))
    val_set = lgb.Dataset(X_train[va_idx], np.log1p(train_df.loc[va_idx, 'price']))
    
    gbm = lgb.train(
        params,
        train_set,
        num_boost_round=2000,
        valid_sets=[val_set],
        callbacks=[lgb.early_stopping(150, verbose=False), lgb.log_evaluation(0)]
    )
    
    # Predict in price space
    oof[va_idx] = np.expm1(gbm.predict(X_train[va_idx]))
    test_pred += np.expm1(gbm.predict(X_test))
    
    fold_smape = smape_np(train_df.loc[va_idx, 'price'], oof[va_idx])
    print(f"  Fold SMAPE: {fold_smape:.4f}, Trees: {gbm.best_iteration}")

test_pred /= 5
cv_smape = smape_np(train_df['price'], oof)
print(f"\n5-Fold CV SMAPE: {cv_smape:.4f}")

# Holdout Validation

In [ ]:
print("\nHoldout Validation...")

X_htr = np.hstack([emb_htr, hold_tr[feature_cols].values]).astype(np.float32)
X_hva = np.hstack([emb_hva, hold_va[feature_cols].values]).astype(np.float32)

train_set = lgb.Dataset(X_htr, np.log1p(hold_tr['price']))
val_set = lgb.Dataset(X_hva, np.log1p(hold_va['price']))

gbm_hold = lgb.train(
    params,
    train_set,
    num_boost_round=2000,
    valid_sets=[val_set],
    callbacks=[lgb.early_stopping(150, verbose=False), lgb.log_evaluation(0)]
)

pred_hold = np.expm1(gbm_hold.predict(X_hva))
holdout_smape = smape_np(hold_va['price'], pred_hold)

print(f"\n{'='*60}")
print(f"HOLDOUT SMAPE: {holdout_smape:.4f}")
print('='*60)

if holdout_smape < 30:
    print("✅ TARGET ACHIEVED!")
elif holdout_smape < 35:
    print("🎯 Very close to target!")
else:
    print("⚠️ Needs improvement")

# Generate Submission

In [ ]:
# Clip to training bounds
lower = train_df['price'].quantile(0.001)
upper = train_df['price'].quantile(0.999)
final_pred = np.clip(test_pred, lower, upper)

submission = pd.DataFrame({
    'sample_id': test_df['sample_id'],
    'price': final_pred
})

submission.to_csv(OUT_DIR/"submission_minimal.csv", index=False)

print(f"\n✅ Submission saved!")
print(f"\nResults:")
print(f"  CV SMAPE:      {cv_smape:.4f}")
print(f"  Holdout SMAPE: {holdout_smape:.4f}")
print("="*60)